In [0]:
# For running on Google Colab:
#
# 1) Open the notebook from GitHub
# 1) Download autoencoder-master.zip from GitHub to your local machine
# 2) Upload to the Google Colab notebook
# 3) Uncomment the following shell commands
# 4) Run the cell
#
#! rm -rf autoencoder-master autoencoder
#! unzip -o autoencoder-master.zip -d .
#! mv autoencoder-master/autoencoder .
#! rm autoencoder-master -rf

In [0]:
import autoencoder.models.quantization
import autoencoder

In [0]:
autoencoder.train()
quant = autoencoder.models.quantization.Quantize()